In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import sys
import dateparser

In [2]:
sys.path.append('..')

In [3]:
import modules.transforms

In [4]:
from modules.transforms import *

In [5]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [6]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [7]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [8]:
for k in ['cuarentenas']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema cuarentenas exists, will not be created


In [9]:
#name='cordones'
#schema='cuarentenas'
#print("creating table "+name+' ,schema: '+schema)
#cordones.to_sql(name, schema=schema,con=cnx,if_exists='replace')
#print("saving table"+path+name+timestamp+'.csv in cache')
#cordones.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

### Cuarentenas

In [10]:
def reshape(df,cut_vars=['REG','CUT','COMUNA'],cut_var='CUT',non_date=['N°',
 'COD REG',
 'REG',
 'ABR',
 'COD PRO',
 'PRO',
 'CUT',
 'COMUNA']):
    cols=df.columns.to_list()
    #print(cols)
    cut=cols.index(cut_var)
    cuts=df[cut_vars].drop_duplicates()
    df=df.T.reset_index()
    df.columns=df.iloc[cut]
    locs=list(range(len(non_date)))
    df=df.drop(locs).rename(columns={cut_var:'Fecha'})

    df=df.melt(id_vars=['Fecha']).dropna()
    idx=[x for x in df.columns if type(x)==int][0]
    print(idx)
    
    df=df.rename(columns={idx:cut_var,'value':'COD_PASO'})
    df=df.merge(cuts,on=cut_var,how='left')
    
    return df

In [11]:
#cuarentenas
url='https://docs.google.com/spreadsheets/d/1WieweYNSPdpmjUIyYcbKp1oaqwlnD61_/export?format=xlsx&gid=103561573'
r = requests.get(url, allow_redirects=True)
open(path+'cuarentenas.xlsx', 'wb').write(r.content)

123679

In [12]:
cuarentenas=pd.read_excel(path+'cuarentenas.xlsx',index_col=None,nrows=348)

<ipython-input-12-589cc3d77ed2>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  cuarentenas=pd.read_excel(path+'cuarentenas.xlsx',index_col=None,nrows=348)


In [13]:
cuarentenas=reshape(cuarentenas,cut_vars=['REG','CUT','COMUNA'])

6


In [14]:
cuarentenas['COD_PASO']=cuarentenas['COD_PASO'].replace({0:np.nan,1:0})

In [15]:
#name='cuarentenas'
#schema='cuarentenas'
#print("creating table "+name+' ,schema: '+schema)
#cuarentenas.to_sql(name, schema=schema,con=cnx,if_exists='replace')
#print("saving table"+path+name+timestamp+'.csv in cache')
#cuarentenas.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

### Paso a Paso

In [16]:
#PASO A PASO 2021
url='https://docs.google.com/spreadsheets/d/1WieweYNSPdpmjUIyYcbKp1oaqwlnD61_/export?format=xlsx&gid=1282224748'
r = requests.get(url, allow_redirects=True)
open(path+'paso_a_paso_2021.xlsx', 'wb').write(r.content)

47547

In [17]:
paso2021=pd.read_excel(path+'paso_a_paso_2021.xlsx',index_col=None,nrows=348)

<ipython-input-17-8939608b1310>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  paso2021=pd.read_excel(path+'paso_a_paso_2021.xlsx',index_col=None,nrows=348)


In [18]:
#PASO A PASO 2020
url='https://docs.google.com/spreadsheets/d/1WieweYNSPdpmjUIyYcbKp1oaqwlnD61_/export?format=xlsx&gid=1670946549'
r = requests.get(url, allow_redirects=True)
open(path+'paso_a_paso_2020.xlsx', 'wb').write(r.content)

142177

In [19]:
paso2020=pd.read_excel(path+'paso_a_paso_2020.xlsx',index_col=None,nrows=348)

<ipython-input-19-f188ad4e4d56>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  paso2020=pd.read_excel(path+'paso_a_paso_2020.xlsx',index_col=None,nrows=348)


In [20]:
pasopaso=pd.concat([reshape(paso2020),reshape(paso2021)])

1
1


In [21]:
cuarentenas=pd.concat([pasopaso,cuarentenas]).dropna(subset=['COD_PASO'])

In [22]:
cuarentenas['Fecha2']=cuarentenas['Fecha']
cuarentenas['Fecha']=pd.to_datetime(cuarentenas['Fecha'],errors='coerce')
fechas_mal=cuarentenas.loc[cuarentenas['Fecha'].isnull(),'Fecha2']
fechas_mal=fechas_mal+'-2020'
cuarentenas.loc[cuarentenas['Fecha'].isnull(),'Fecha']=fechas_mal.apply(lambda x: dateparser.parse(x))

/opt/conda/lib/python3.8/site-packages/tzlocal/unix.py:158: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn('Can not find any timezone configuration, defaulting to UTC.')


In [23]:
cuarentenas['CUT']=cuarentenas['CUT'].astype(str).str.strip()

In [24]:
comunas=pd.read_sql_table('comunas', schema='geo',con=cnx)
comunas=comunas[['T_POB','COMUNA']].rename(columns={'COMUNA':'CUT'})
comunas['CUT']=comunas['CUT'].astype(str).str.strip()

/opt/conda/lib/python3.8/site-packages/sqlalchemy/dialects/postgresql/base.py:3198: SAWarning: Did not recognize type 'geometry' of column 'geometry'
  util.warn(


In [25]:
cuarentenas=cuarentenas.merge(comunas,on='CUT',how='left')

In [26]:
cuarentenas.to_sql('cuarentenas', schema='cuarentenas',con=cnx,if_exists='replace')

In [27]:
cuarentenas['REG'].unique()

array(['Tarapacá', 'Antofagasta', 'Atacama', 'Coquimbo', 'Valparaíso',
       "Libertador General Bernardo O'Higgins", 'Maule', 'Biobío',
       'La Araucanía', 'Los Lagos',
       'Aysén del General Carlos Ibáñez del Campo',
       'Magallanes y de la Antártica Chilena',
       'Metropolitana de Santiago', 'Los Ríos', 'Arica y Parinacota',
       'Ñuble'], dtype=object)

In [28]:
cuarentenas

,Fecha,CUT,COD_PASO,REG,COMUNA,Fecha2,T_POB
0,2020-07-24,1101,1,Tarapacá,Iquique,2020-07-24 00:00:00,191468.0
1,2020-07-25,1101,1,Tarapacá,Iquique,2020-07-25 00:00:00,191468.0
2,2020-07-26,1101,1,Tarapacá,Iquique,2020-07-26 00:00:00,191468.0
3,2020-07-27,1101,1,Tarapacá,Iquique,2020-07-27 00:00:00,191468.0
4,2020-07-28,1101,1,Tarapacá,Iquique,2020-07-28 00:00:00,191468.0
...,...,...,...,...,...,...,...
71279,2020-04-18,16103,0.0,Ñuble,Chillán Viejo,18-abr,30907.0
71280,2020-04-19,16103,0.0,Ñuble,Chillán Viejo,19-abr,30907.0
71281,2020-04-20,16103,0.0,Ñuble,Chillán Viejo,20-abr,30907.0
71282,2020-04-21,16103,0.0,Ñuble,Chillán Viejo,21-abr,30907.0


In [30]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_cuarentenas.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/Cuarentenas.ipynb')

0